# Time Travel Debugger project

## Personal Information

In [1]:
PROJECT_TYPE = 2
NAME = ["Daniel Gusenburger", "Daniel Tabellion"]
ID = ["2544941", "2555742"]
IMPLEMENTED = set()

In [2]:
print(f"Implemented features: {IMPLEMENTED}")

## Implementation

In [3]:
from time_travel_debugger.view.gui import GUI
from time_travel_debugger.view.cli import TimeTravelCLI
from main import remove_html_markup

    
with GUI():
    remove_html_markup("<tag>hallo</tag>")

Play(value=0)

IntSlider(value=1, description='Playback Speed', max=5, min=1)

ToggleButton(value=False, icon='history', tooltip='Reverse autoplay')

HTML(value='\n            <style>\n               .jupyter-widgets-output-area .output_scroll {\n             …

GridspecLayout(children=(HBox(children=(VBox(children=(Button(description='Backstep', icon='step-backward', st…

## Presentation

## Summary